<a href="https://colab.research.google.com/github/RKDash7/XAI-Enhanced-YOLOv8-for-Transparent-and-Interpretable-Object-Detection-in-Critical-Applications/blob/main/Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

'''
model = YOLO('yolov8n.pt')  # load model YOLO or SSD or Faster R-CNN
results = model('your_image.jpg')  # run inference
'''
# If processing a single image, results is a Results object
# For multiple images, results will be a list of Results objects

# Access detections for the first (or only) image
detections1 = results[0].boxes  # 'boxes' contains the detection data

# Get detection data: tensor with shape [num_detections, 6] (xmin, ymin, xmax, ymax, confidence, class)
detections_tensor = detections1.data  # or detections.xyxy

# Total detections
total_detections = len(detections_tensor)

# Set confidence threshold
conf_threshold = 0.75

# Filter detections based on confidence
filtered_detections = detections_tensor[detections_tensor[:, 4] >= conf_threshold]

# Calculate drop percentage
if total_detections > 0:
    drop_percentage = (1 - len(filtered_detections) / total_detections) * 100
else:
    drop_percentage = 0

print(f'Drop percentage: {drop_percentage:.2f}%')

In [ ]:


# Assuming you have:
# - results: model inference results for an image
# - ground_truth_box: [xmin, ymin, xmax, ymax]

# Get detections
detections = results[0].boxes
detections_data = detections.data

# Find top detection by confidence
# Move tensor to CPU before converting to numpy
top_detection = detections_data[np.argmax(detections_data[:, 4].cpu().numpy())]

# Move top_detection to CPU before converting to numpy if needed
xmin, ymin, xmax, ymax = top_detection[:4].cpu().numpy()
confidence = top_detection[4].item()

# Compute center point
center_x = (xmin + xmax) / 2
center_y = (ymin + ymax) / 2

# Check if center point is inside ground truth box
# This check requires ground_truth_box to be defined elsewhere in the notebook
# if (xmin <= center_x <= xmax) and (ymin <= center_y <= ymax):
#     print("Hit")
# else:
#     print("Miss")

# Note: The original code attempts to check if the center of the *predicted* bounding box is inside the *predicted* bounding box itself, which will always be true.
# It seems like the intention might have been to compare the predicted box to a ground truth box.
# If you have ground truth bounding box information, replace the commented-out check with a comparison to the ground truth box.

print(f"Top detection confidence: {confidence:.2f}")
print(f"Top detection bounding box (xmin, ymin, xmax, ymax): {xmin:.2f}, {ymin:.2f}, {xmax:.2f}, {ymax:.2f}")
print(f"Top detection center (x, y): {center_x:.2f}, {center_y:.2f}")

In [ ]:


def load_ground_truth(image_file, labels_dir='/content/lung--1/valid/labels/', img_width=640, img_height=480):
    """
    Load ground truth boxes and classes for the given image.

    Args:
        image_file (Path): Path to the image file.
        labels_dir (str): Directory containing label files.
        img_width (int): Width of the image.
        img_height (int): Height of the image.

    Returns:
        gt_boxes (list of [xmin, ymin, xmax, ymax]): Bounding boxes in pixel coordinates.
        gt_classes (list of int): Class labels.
    """
    label_file = Path(labels_dir) / (image_file.stem + '.txt')
    gt_boxes = []
    gt_classes = []

    if not label_file.exists():
        return gt_boxes, gt_classes  # No ground truth for this image

    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])

            # Convert normalized coordinates to pixel values
            xmin = int((x_center - width / 2) * img_width)
            ymin = int((y_center - height / 2) * img_height)
            xmax = int((x_center + width / 2) * img_width)
            ymax = int((y_center + height / 2) * img_height)

            gt_boxes.append([xmin, ymin, xmax, ymax])
            gt_classes.append(class_id)
    return gt_boxes, gt_classes

In [ ]:
def compute_iou(box1, box2):
    """
    Compute the Intersection over Union (IoU) of two bounding boxes.

    Args:
        box1 (list or array): [xmin, ymin, xmax, ymax]
        box2 (list or array): [xmin, ymin, xmax, ymax]

    Returns:
        float: IoU value between 0 and 1.
    """
    xmin1, ymin1, xmax1, ymax1 = box1
    xmin2, ymin2, xmax2, ymax2 = box2

    # Calculate intersection coordinates
    inter_xmin = max(xmin1, xmin2)
    inter_ymin = max(ymin1, ymin2)
    inter_xmax = min(xmax1, xmax2)
    inter_ymax = min(ymax1, ymax2)

    # Compute area of intersection
    inter_width = max(0, inter_xmax - inter_xmin)
    inter_height = max(0, inter_ymax - inter_ymin)
    inter_area = inter_width * inter_height

    # Compute areas of each box
    area1 = (xmax1 - xmin1) * (ymax1 - ymin1)
    area2 = (xmax2 - xmin2) * (ymax2 - ymin2)

    # Compute union area
    union_area = area1 + area2 - inter_area

    # Avoid division by zero
    if union_area == 0:
        return 0

    # Calculate IoU
    iou = inter_area / union_area
    return iou

In [ ]:

# Path to dataset images and labels
images_path = Path('/content/lung--1/valid/images/')
labels_path = Path('/content/lung--1/valid/labels/')  # if available

# Initialize counters
total_images = 0
correct_detections = 0

# IoU threshold for considering a detection correct
iou_threshold = 0.05

# Loop through images
for image_file in images_path.glob('*.jpg'):
    total_images += 1
    # Run inference
    results1 = model(str(image_file))
    preds = results1[0]  # predictions: (xmin, ymin, xmax, ymax, confidence, class)

    # Load ground truth boxes and classes for this image
    # You need to implement loading your labels here
    gt_boxes, gt_classes = load_ground_truth(image_file)

    # For simplicity, assume 1 primary object per image
    if len(gt_boxes) == 0:
        print(f"No ground truth for {image_file}")
        continue  # skip if no ground truth

    # Check if any predicted box matches ground truth
    detection_matched = False
    # Access detection data from the 'boxes' attribute
    if preds.boxes: # Check if any boxes were detected
      for pred in preds.boxes.data:
          pred_box = pred[:4].cpu().numpy()
          pred_cls = int(pred[5].cpu().item())

          for gt_box, gt_cls in zip(gt_boxes, gt_classes):
              iou = compute_iou(pred_box, gt_box)
              if iou >= iou_threshold and pred_cls == gt_cls:
                  detection_matched = True
                  break
          if detection_matched:
              break

    if detection_matched:
        correct_detections += 1

accuracy = correct_detections / total_images if total_images > 0 else 0
print(f"Detection Accuracy: {accuracy:.2f}")

In [ ]:
# Assuming 'model' is your trained YOLOv8 model and 'fig1' contains test image paths
# Initialize the YOLOv8LRP explainer
lrp = YOLOv8LRP(model, power=2, eps=1e-05, device='cpu') # Use CPU as per previous attempts

# Load and preprocess the image
image_path = fig1[3] # Use one of the test images
image = Image.open(image_path)
desired_size = (512, 640)
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(desired_size),
    torchvision.transforms.ToTensor(),
])
image_tensor = transform(image).to('cpu').float()

# Compute the explanation
explanation_lrp = lrp.explain(image_tensor, cls='Consolidation', contrastive=False).cpu() # Assuming 'Consolidation' is a class in your model

# Compute metrics
avg_drop = AverageDrop(lrp)
increase_confidence = IncreaseConfidence(lrp)
pointing_game = PointingGame(lrp)

# Calculate and print the metrics
avg_drop_score = avg_drop(image_tensor, explanation_lrp, cls='Consolidation')
increase_confidence_score = increase_confidence(image_tensor, explanation_lrp, cls='Consolidation')
pointing_game_score = pointing_game(image_tensor, explanation_lrp, cls='Consolidation')


print(f"Average Drop % for 'Consolidation': {avg_drop_score}")
print(f"Increase in Confidence for 'Consolidation': {increase_confidence_score}")
print(f"Pointing Game for 'Consolidation': {pointing_game_score}")